# Quick Start: Opik + OpenAI + DotEnv

This notebook demonstrates how to:
1. Load environment variables using `python-dotenv`.
2. Initialize OpenAI client.
3. Integrate Opik for tracing.
4. Use Opik for Prompt Management.

In [1]:
# 1. Environment Setup
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded.")

Environment variables loaded.


In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
import opik

opik.configure(use_local=True)

Please enter your Opik instance URL: http://localhost:5173/default/home


OPIK: Opik is not accessible at http://localhost:5173/. Please try again, the URL should follow a format similar to http://localhost:5173/


Please enter your Opik instance URL: http://localhost:5173/


OPIK: Opik is not accessible at http://localhost:5173/. Please try again, the URL should follow a format similar to http://localhost:5173/


In [8]:
# 2. Opik Setup & Tracing
import opik
from opik.integrations.openai import track_openai
from openai import OpenAI

# Ensure Opik points to the host instance (using the URL override from docker-compose)
# docker-compose.yml sets OPIK_URL_OVERRIDE=http://host.docker.internal:5173/api
# We can verify this:
print(f"Opik URL: {os.getenv('OPIK_URL_OVERRIDE')}")

# Initialize OpenAI client
client = OpenAI()

# Wrap the client with Opik tracker
openai_client = track_openai(client)

print("Opik configured and OpenAI client wrapped.")

Opik URL: None
Opik configured and OpenAI client wrapped.


In [ ]:
# 3. Run a Trace
try:
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Write a haiku about observability."}
        ]
    )
    print("\nResponse:")
    print(completion.choices[0].message.content)
    print("\nCheck your Opik dashboard for the trace!")
except Exception as e:
    print(f"Error making request: {e}")

In [ ]:
# 4. Prompt Management
try:
    # Create or retrieve a prompt
    prompt_name = "observability-story-1"
    prompt_template = "Write a short story about {{topic}} using {{adjective}} language."
    
    prompt = opik.Prompt(
        name=prompt_name,
        prompt=prompt_template
    )
    
    print(f"\nPrompt '{prompt.name}' created/retrieved.")
    print(f"Template: {prompt.prompt}")
    
    # Format the prompt
    formatted_prompt = prompt.format(topic="AI agents", adjective="flowery")
    print(f"\nFormatted Prompt:\n{formatted_prompt}")
    
    # Use the formatted prompt in a call (traced)
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": formatted_prompt}]
    )
    
    print("\nStory Response:")
    print(completion.choices[0].message.content)
    
except Exception as e:
    print(f"Error in prompt management: {e}")